# RAG(Retrieval-Augmented-Generation) Client

## 1. Download Model

```bash
gai pull instructor-sentencepiece
```

## 2. Start RAG Service

* Go to /gai

* Open docker-compose.yml

* Make sure that the following section is not commented.

```yaml
    gai-rag-svr:
        image: kakkoii1337/gai-rag-svr:1.0.17
        container_name: gai-rag-svr
        ...
```

* Run the command:

```sh
docker-compose up
```



## Example 1: Ask about for a Web Page

### Step 1: Scrape text from the web for testing

Scrape a speech from the web and save it to a file.

In [1]:
import os
os.environ["LOG_LEVEL"]="INFO"
from gai.lib.tools.scraper import Scraper
scraper=Scraper()
text,links = scraper.scrape("https://www.pmo.gov.sg/Newsroom/national-day-rally-2023")
import tempfile
with tempfile.NamedTemporaryFile(mode="w", delete=False) as f:
    f.write(text)
    file_path = f.name

### Step 2: Index the file

In [2]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Index the sample speech
result = await rag.index_document_async(
    collection_name="demo",
    file_path=file_path,
    file_type="txt",
    title="2023 National Day Rally Speech",
    source="https://www.pmo.gov.sg/Newsroom/national-day-rally-2023",
    async_callback=async_callback,
)


INFO     Connected to ws://localhost:12036/gen/v1/rag/index-file/ws/demo


async_callback: {'message': 'Request received.'}
async_callback: {'message': 'Breaking down document into chunks ...'}
async_callback: {'message': 'Start indexing...'}
async_callback: {'progress': 2}
async_callback: {'progress': 4}
async_callback: {'progress': 6}
async_callback: {'progress': 8}
async_callback: {'progress': 10}
async_callback: {'progress': 12}
async_callback: {'progress': 14}
async_callback: {'progress': 16}
async_callback: {'progress': 18}
async_callback: {'progress': 20}
async_callback: {'progress': 22}
async_callback: {'progress': 24}
async_callback: {'progress': 26}
async_callback: {'progress': 28}
async_callback: {'progress': 30}
async_callback: {'progress': 32}
async_callback: {'progress': 34}
async_callback: {'progress': 36}
async_callback: {'progress': 38}
async_callback: {'progress': 40}
async_callback: {'progress': 42}
async_callback: {'progress': 44}
async_callback: {'progress': 46}
async_callback: {'progress': 48}
async_callback: {'progress': 51}
async_callb

### Step 3. Asking a question

In [3]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

# Retrieve from query
results = await rag.retrieve_async(
    collection_name="demo",
    query_texts="Who are the young seniors?",
    n_results=1
)
for result in results:
    print(result)


{'documents': 'and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired. Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,

### 3. List Collections

In [4]:
# Load RAG client
from gai.rag.client.rag_client_async import RagClientAsync
rag = RagClientAsync({
        "type": "rag",
        "url": "http://localhost:12036/gen/v1/rag",
        "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws"
})

await rag.list_collections_async()

{'collections': ['demo']}

### 4. List Document Headers

In [5]:
docs = await rag.list_documents_async(collection_name="demo")
docs

[IndexedDocPydantic(Id='HjXcCqm-IL_9pdam6K8_Qcr7xvqxwVrYNRHlCRDeX6g', CollectionName='demo', ByteSize=44189, FileName='tmpaclyrqnl', FileType='txt', File=None, Source='https://www.pmo.gov.sg/Newsroom/national-day-rally-2023', Abstract=None, Authors='', Title='2023 National Day Rally Speech', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2024, 10, 28, 10, 13, 59, 950605), UpdatedAt=datetime.datetime(2024, 10, 28, 10, 13, 59, 950614), IsActive=True, ChunkGroups=[IndexedDocChunkGroupPydantic(Id='b62ccaa6-87d3-43eb-9282-ee252feb29bc', DocumentId='HjXcCqm-IL_9pdam6K8_Qcr7xvqxwVrYNRHlCRDeX6g', SplitAlgo='recursive_split', ChunkCount=49, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/0553499d446a406a81ccd17f03c3a253')])]

### 5. Download File

In [6]:
await rag.get_document_file_async(collection_name="demo",document_id=docs[0].Id)

---

## Appendix: Multi-Step Indexing

This achieves the same outcome as step 2 but broken down into multiple steps for ease of debugging. For this demo, we shall use a PDF file.

### Step 0: Setup

In [1]:
import os
from gai.rag.client.rag_client_async import RagClientAsync

rag = RagClientAsync({
    "type": "rag",
    "url": "http://localhost:12036/gen/v1/rag",
    "ws_url": "ws://localhost:12036/gen/v1/rag/index-file/ws",
})


### Step 1: Create header

In [2]:
here = os.path.dirname(__name__)
file_path=os.path.join(here,"ReAct-2210.03629.pdf")
doc_header = await rag.step_header_async(
    collection_name="demo",
    file_path=file_path,
    title="ReAct: Synergizing Reasoning and Acting in Language Models",
    source="https://arxiv.org/abs/2210.03629",
)
doc_header

IndexedDocPydantic(Id='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', CollectionName='demo', ByteSize=633805, FileName='ReAct-2210.03629.pdf', FileType='.pdf', File=None, Source='https://arxiv.org/abs/2210.03629', Abstract=None, Authors='', Title='ReAct: Synergizing Reasoning and Acting in Language Models', Publisher='', PublishedDate=None, Comments='', Keywords='', CreatedAt=datetime.datetime(2024, 10, 28, 10, 57, 42, 969178), UpdatedAt=datetime.datetime(2024, 10, 28, 10, 57, 42, 969187), IsActive=True, ChunkGroups=[])

### Step 2: Split Document

In [3]:
chunkgroup = await rag.step_split_async(
    collection_name="demo",
    document_id=doc_header.Id,
    chunk_size=1000,
    chunk_overlap=100
)
chunkgroup

IndexedDocChunkGroupPydantic(Id='74dd9287-74cd-4f37-9d49-7616660793c7', DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', SplitAlgo='recursive_split', ChunkCount=114, ChunkSize=1000, Overlap=100, IsActive=True, ChunksDir='/tmp/chunks/89141cf2760b4dc888f4d8da4eff444d')

### Step 3: Index Chunks

In [4]:
# Create callback
import json
async def async_callback(text):
    jsoned = json.loads(text)
    print(f"async_callback: {jsoned}")

# Spin off listener
from gai.lib.common.StatusListener import StatusListener
ws_url = f"ws://localhost:12036/gen/v1/rag/index-file/ws/demo"
listener = StatusListener(ws_url)
import asyncio
listen_task=asyncio.create_task(listener.listen(async_callback))

result = await rag.step_index_async(
    collection_name="demo",
    document_id=chunkgroup.DocumentId,
    chunkgroup_id=chunkgroup.Id,
    async_callback=async_callback
)
result


async_callback: {'progress': 0}
async_callback: {'progress': 1}
async_callback: {'progress': 2}
async_callback: {'progress': 3}
async_callback: {'progress': 4}
async_callback: {'progress': 5}
async_callback: {'progress': 6}
async_callback: {'progress': 7}
async_callback: {'progress': 7}
async_callback: {'progress': 8}
async_callback: {'progress': 9}
async_callback: {'progress': 10}
async_callback: {'progress': 11}
async_callback: {'progress': 12}
async_callback: {'progress': 13}
async_callback: {'progress': 14}
async_callback: {'progress': 14}
async_callback: {'progress': 15}
async_callback: {'progress': 16}
async_callback: {'progress': 17}
async_callback: {'progress': 18}
async_callback: {'progress': 19}
async_callback: {'progress': 20}
async_callback: {'progress': 21}
async_callback: {'progress': 21}
async_callback: {'progress': 22}
async_callback: {'progress': 23}
async_callback: {'progress': 24}
async_callback: {'progress': 25}
async_callback: {'progress': 26}
async_callback: {'pro

IndexedDocChunkIdsPydantic(DocumentId='Mm1GAmuGfs3VoA2sit0njabQGAZ75eyfszIkAxGY5e4', ChunkgroupId='74dd9287-74cd-4f37-9d49-7616660793c7', ChunkIds=['46ecd90f-6ab3-4828-ad42-a04f09de5f0f', '40aaad50-ad2d-4225-8875-8662c777da79', '28127482-86aa-4782-9474-c3a09c572aa8', 'e3983bcf-da83-4bb6-9397-057a3f795ee9', '9467ee8e-39b2-496d-9eae-892c5acbcb97', '9e3db04a-99ea-4ca1-b21d-7d4150a6ab49', '872ebe50-8616-41f0-be0a-4d09694b0593', 'ce7aa147-b6f5-4d15-826f-7d0ddc42a7d4', 'a60d4c7a-8852-4072-a1dd-278525d9dcc4', 'ebe13569-14ee-42ae-bc15-679d3b72ae2e', '9b2afce2-97bf-4a6c-8d76-b1235689ec66', '55a2ed8b-f232-4b08-b56f-b00a82bfe3cf', 'abcb0085-e6a1-42bd-8056-7adbb3b1eeda', '2571d06d-4496-48ef-90f7-42fb8e6d60b6', '314884ed-9a83-4a04-a313-a45b8f603fcd', '77cf982a-9fdd-488a-bc42-46f73cf49389', '1bfed1da-49bf-4459-9190-f7a2b1269b99', '187d774b-f4fa-4dc6-a18b-13140f8f3265', '29fe75c3-0f6e-426e-9c56-0ba7fc382c10', 'bfb8122d-cd9d-4b2c-98bd-08a2208816f5', 'ebc1fcb2-85d5-4393-84e9-922f4f256e84', '48752e4c-be

async_callback: {'progress': 100}
